In [ ]:
def signals_sec2(Noversampl, depth0, oof, ofcont, FILT, Order, FiltType, FILTFACT, AutoPhase, ph0, IQ, noise):

    sstep = nptslambda/Noversampl
    depth = depth0 * 1e-6
    #ph0 = ph0/2/np.pi
    ph = int(nptslambda * ph0) #ph0 between 0 - 1
    
    #FRINGE
    er = np.sqrt(amp) #reference field
    es = R*np.sqrt(amp)*np.exp(1j*(2*np.pi*oof*(2*depth/c))) #sample field
    
    #I/Q detection and background subtraction
    if BG:
        sfI = np.square(np.abs(er + es)) - (np.square(er) + np.square(R*np.sqrt(amp)))
        sfQ = np.square(np.abs(er + es*np.exp(1j*np.pi/2)))  - (np.square(er) + np.square(R*np.sqrt(amp)))
    else:
        sfI = np.square(np.abs(er + es))
        sfQ = np.square(np.abs(er + es*np.exp(1j*np.pi/2)))
    
    #add noise
    sfI = sfI + np.random.randint(-10000,10000,len(oof))/10000*noise
    sfQ = sfQ + np.random.randint(-10000,10000,len(oof))/10000*noise

    if IQ:
        sf = sfI + 1j*sfQ
    else:
        sf = sfI    
    
    #ANALOG FILTER
    if FILT:
        if FiltType == 1:
            b,a = signal.butter(Order, FILTFACT/nptslambda, 'low') 
        else:
            b,a = signal.cheby1(Order,0.5,FILTFACT/nptslambda,'low')

        sfilt = signal.filtfilt(b,a,sf);
        
    else:
        sfilt = sf
        
    #SAMPLING
    #cropping the middle one of three sweep
    ind = int(len(amp)/3)
    amp0 = amp[ind+1:ind+1+npts]; 

    oof = oof[ind+1:ind+1+npts];
    ofcont = ofcont[ind+1:ind+1+npts];
   
    inds = int(ind+1)
    inde = int(ind+1+npts)
    indstep = int(sstep)
    ss = sfilt[inds:inde:indstep] # DAQ sampling
   
    ##DEPTH VECTOR
    #inds = 0
    #inde = int(len(ofcont))
    #indstep = int(sstep)
    #ofcont_sampled = ofcont[inds:inde:indstep]            #%sampled frequency vector
    #df_sampled = ofcont_sampled[1]-ofcont_sampled[0]      #frequency resolution of sampled field
    #dt_sampled = 1/(df_sampled*nfft);
    #temp_vec = np.linspace(-nfft/2,nfft/2,nfft)
    #dsampled = temp_vec*dt_sampled*c/2                    #depth vector sampled sampled fringe
    
    #FFT
    Ss0 = np.fft.fftshift(np.fft.ifft(ss, nfft)/Noversampl)
    Ssout = Ss0
    Ss = np.square(np.abs(Ssout))
 
    
    return(Ss)